In [ ]:
import mlflow

# we never defined an official name, make sure you use the name of your own registered model here
model_name = "mushroom"

# this also only works if you set this alias for above model
model_version_alias = "champion"

model_uri = f"models:/{model_name}@{model_version_alias}"
model = mlflow.sklearn.load_model(model_uri)

For the future: we have to convert the dict to whatever sklearn accepts *dataframe, or (1, 8) shaped numpy array. Dataframe is easier here.

In [ ]:
import pandas as pd
import numpy as np

d = {'cap-diameter': 800.0, 'cap-shape': 5.0, 'gill-attachment': 4.0, 'gill-color': 0.0, 'stem-height': 1.1368282303, 'stem-width': 904.0, 'stem-color': 11.0, 'season': 0.7557389093}
model.predict(pd.DataFrame(d, index=[0]))

# Quix

## Setup

In [63]:
from quixstreams import Application

# create main quix object
app = Application(
    broker_address="message-broker:9092",
)

# define topic and message format
input_topic = app.topic(name="mushroom_inference_request", value_deserializer="json")

# create a StreamingDataFrame
sdf = app.dataframe(topic=input_topic)

## Print message

In [ ]:
#sdf = sdf.update(lambda message: print(f"Inference Request: {message}"))
sdf = sdf.update(print)

app.run(sdf)

## Call model

Verbose:

In [60]:
if False:
    def predict(d: dict) -> int:
        # reads single row dict and returns prediction as integer scalar
        # spelled out to make it easier to read
        
        input_as_df = pd.DataFrame(d, index=[0])
        pred_as_array = model.predict(input_as_df)
        pred_as_scalar = pred_as_array[0]
        
        return int(pred_as_scalar)
    
    sdf.update(lambda m: m.update({'prediction': predict(m)}))
    sdf = sdf.update(print)
    
    app.run(sdf)

[2024-09-02 11:33:13,729] [INFO] [quixstreams] : Stop processing of StreamingDataFrame


In [64]:
sdf = sdf.update(lambda m: m.update({'prediction': int(model.predict(pd.DataFrame(m, index=[0]))[0])}))
sdf = sdf.update(print)

output_topic = app.topic(name="mushroom_inference_result", value_deserializer="json")

sdf.to_topic(output_topic)

app.run(sdf)

[2024-09-02 11:42:40,402] [INFO] [quixstreams] : Starting the Application with the config: broker_address="{'bootstrap.servers': 'message-broker:9092'}" consumer_group="quixstreams-default" auto_offset_reset="latest" commit_interval=5.0s commit_every=0 processing_guarantee="at-least-once"
[2024-09-02 11:42:40,407] [INFO] [quixstreams] : Topics required for this application: "mushroom_inference_request", "mushroom_inference_result"
[2024-09-02 11:42:40,415] [INFO] [quixstreams] : Creating a new topic "mushroom_inference_result" with config: "{'num_partitions': 1, 'replication_factor': 1, 'extra_config': {}}"
[2024-09-02 11:42:41,421] [INFO] [quixstreams] : Topic "mushroom_inference_result" has been created
[2024-09-02 11:42:41,423] [INFO] [quixstreams] : Validating Kafka topics exist and are configured correctly...
[2024-09-02 11:42:41,435] [INFO] [quixstreams] : Kafka topics validation complete
[2024-09-02 11:42:41,439] [INFO] [quixstreams] : Initializing state directory at "/notebooks

{'cap-diameter': 1003.0, 'cap-shape': 5.0, 'gill-attachment': 4.0, 'gill-color': 6.0, 'stem-height': 0.0099135591, 'stem-width': 1608.0, 'stem-color': 10.0, 'season': 1.002343729, 'prediction': 1}
{'cap-diameter': 574.0, 'cap-shape': 4.0, 'gill-attachment': 0.0, 'gill-color': 9.0, 'stem-height': 0.1512698851, 'stem-width': 617.0, 'stem-color': 11.0, 'season': 1.7742273406, 'prediction': 0}
{'cap-diameter': 1355.0, 'cap-shape': 0.0, 'gill-attachment': 0.0, 'gill-color': 8.0, 'stem-height': 1.1622464745, 'stem-width': 2474.0, 'stem-color': 8.0, 'season': 0.5996043647, 'prediction': 0}


[2024-09-02 13:05:52,153] [INFO] [quixstreams] : REQTMOUT [rdkafka#consumer-55] [thrd:message-broker:9092/bootstrap]: message-broker:9092/0: Timed out FetchRequest in flight (after 3501284ms, timeout #0)
[2024-09-02 13:05:51,815] [WARNING] [quixstreams] : REQTMOUT [rdkafka#consumer-55] [thrd:message-broker:9092/bootstrap]: message-broker:9092/0: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
[2024-09-02 13:05:51,830] [ERROR] [quixstreams] : FAIL [rdkafka#consumer-55] [thrd:message-broker:9092/bootstrap]: message-broker:9092/0: 1 request(s) timed out: disconnect (after 4990762ms in state UP)
[2024-09-02 13:05:51,847] [WARNING] [quixstreams] : SESSTMOUT [rdkafka#consumer-55] [thrd:main]: Consumer group session timed out (in join-state steady) after 3501213 ms without a successful response from the group coordinator (broker 0, last error was Success): revoking assignment and rejoining group
[2024-09-02 13:05:51,855] [ERROR] [quixstreams] : Kafka consumer err

In [ ]:
import isort

print(isort.code("""
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.neighbors import KernelDensity
from random import randrange
import pandas as pd
import time
import sys
import argparse
import time
from loguru import logger
"""))